# Spark Data Processing

Spark has two different types of operations Transformations and Actions.

## Transformations 
- Operations that create a new RDD, usually based on a previous one. 
- Does not evaluate the expression until an action is called (lazy evaluation).
- Spark is able to infer the output type.
- You can concatenate multiple transformations, before an action.
    
There are two types of transformations:

<img src="images/transformation_types.png" title="Spark transformation types" width="700px"/>

### **Narrow Transformations**

Some examples of narrow transformations are:

#### **Map**

A given function is applied for each pair of key-value to generate an intermediate key-value. You can combine multiple Map functions.

**In simple terms**

Imagine you have a list of items, and for each item, you want to perform a specific operation or transformation. The \"map\" operation takes each item in the list, applies a function to it, and produces a new list with the transformed items.

In [ ]:
?rdd.map

**Example**

In [ ]:
doubled_rdd = rdd.map(lambda x: x * 2)
doubled_rdd.collect()

#### **Filter**
Return a new RDD containing only the elements that satisfy a predicate.

In [ ]:
?rdd.filter

**Example**

In [ ]:
even_rdd = rdd.filter(lambda x: x % 2 == 0)
even_rdd.collect()

### **Wide Transformations**

This transformations need to use data from other partitions and, thus, perform a Shuffle

#### **The Shuffle Process**

It's the process of redistributing the data across the partitions. This can involve reorganizing the data within a single machine or moving data across multiple machines in the cluster. Shuffling can be expensive in terms of performance because it involves disk I/O, data serialization, and network I/O.

**In simple terms**

Imagine you're a school teacher, and you've given each student in your class a card with a number on it. Initially, the students are seated randomly. Now, you want to group them based on the number they hold, say, all students holding even numbers on one side and those with odd numbers on the other.

To do this, students would have to get up, move around, and find their new positions based on their card numbers. This process of rearranging is analogous to the "shuffle" operation in Spark.

**Why does shuffle happen?**

Shuffle usually occurs during operations that require data reorganization. Common operations in Spark that cause shuffling include:

- **groupBy**: Grouping data by certain keys.
- **reduceByKey**: Reducing data by key.
- **join**: Joining two datasets based on keys.

**Why is shuffle important?**

Understanding shuffle is essential because:

- **Performance Implications**: Shuffling can be a performance bottleneck. It involves writing data to disk, transferring data over the network, and reading data back into memory. If you're aware of when shuffling occurs, you can potentially optimize your Spark jobs to minimize shuffling.

- **Resource Management**: Shuffling can consume a significant amount of resources. Knowing when and why shuffling is happening can help in tuning the Spark configuration and resources appropriately.

Some examples of wide transformations are:

#### **GroupBy**
Groups items by a condition

In [ ]:
?rdd.groupBy

**Example**

In [ ]:
even_odd_groups_rdd = rdd.groupBy(lambda x: x % 2 == 0)
[[elem[0], list(elem[1])] for elem in even_odd_groups_rdd.collect()]

#### **Repartition**
Rearranges the RDD to match the new number of partitions with equal size of partitions.

In [ ]:
?rdd.repartition

**Example**

In [ ]:
rdd_1_partitions = rdd.repartition(1)
rdd_3_partitions = rdd.repartition(3)
print(rdd.glom().collect())
print(rdd_1_partitions.glom().collect())
print(rdd_3_partitions.glom().collect())

#### **Coalesce**
Rearranges the RDD to match the new number of partitions with equal size of partitions. If `shuffle` is `False`, you can only reduce the number of partitions and the transformation will be **narrow**.

In [ ]:
?rdd.coalesce

**Example**

In [ ]:
rdd_1_partitions = rdd.coalesce(1)
rdd_3_partitions = rdd.coalesce(3)
print(rdd.glom().collect())
print(rdd_1_partitions.glom().collect())
print(rdd_3_partitions.glom().collect())

## Actions
- Operations that evaluates all the transformations defined.
- Forces the evaluation to save or use the result data.

Some examples of actions are:

#### **Reduce**

A combination function that groups each key to calculate the aggregation of the multiple values associated to the key.

**In simple terms**

After you've transformed your list using "map", you might want to combine these items in some way to produce a single result. That's where "reduce" comes in. "Reduce" takes the list and applies a function that combines two items at a time, repeatedly, until only one item (a single result) remains.

It's like folding a long piece of paper: you take two adjacent sections, fold them together, then fold the resulting piece with the next section, and so on, until you're left with a small, folded chunk.

In [ ]:
?rdd.reduce

**Example**: Using the list of doubled numbers from before, \\([2, 4, 6, 8, 10]\\), let's say you want to find their sum. Using a \"reduce\" operation, you'd combine two numbers at a time until you get the total sum: \\(2 + 4 = 6\\), \\(6 + 6 = 12\\), \\(12 + 8 = 20\\), and \\(20 + 10 = 30\\). The final result is \\(30\\).

In [ ]:
def add(x, y):
    res = x + y
    print(f'{x} + {y} = {res}')
    return res

sum_result = doubled_rdd.reduce(add)
sum_result

#### **Fold**

Applies the reduction function combining elements together, but including a zero value for each reduction step (partition=.

**In simple terms**

Is the same as the Reduce function but also recieves an initial value called `zeroValue`

In [ ]:
?rdd.fold

**Example**

In [ ]:
doubled_rdd.glom().collect()

In [ ]:
fold_result = doubled_rdd.fold(3, add)
fold_result

In [ ]:
fold_result = doubled_rdd.coalesce(1).fold(3, add)
fold_result

In the following image you can see the process of counting words for a list of sentences performing Map, Shuffle and Reduce.

<img src="images/map_reduce_shuffle_operation.png" title="Map, Reduce and Shuffle Operation" width="700px"/>

## Exercises

1. Create a SparkSession
2. Load the provided list of Strings into an RDD
3. Count the words in each sentence
4. Filter the sentences with less than 5 words
5. Print the sum of all the words

In [ ]:
str_list = ["Spark is such a cool piece of software", "I love Python", "The MapReduce model was revolutionary", "I like dogs"]

# ---- INSERT CODE HERE ----

**Solution**:

In [ ]:
from pyspark.sql import SparkSession

str_list = ["Spark is such a cool piece of software", "I love Python", "The MapReduce model was revolutionary", "I like dogs"]

spark = SparkSession.builder.getOrCreate()

rdd = spark.sparkContext.parallelize(str_list)
word_count_rdd = rdd.map(lambda s: len(s.split()))
filtered_word_count_rdd = word_count_rdd.filter(lambda x: x >= 5)
words_sum = filtered_word_count_rdd.reduce(lambda n1, n2: n1 + n2)
words_sum